# Even reduction

In [1]:
using DynamicPolynomials
@polyvar x

(x,)

We would like to find the minimum value of the following polynomial:

In [2]:
poly = x^4 - 2x^2

using SumOfSquares

We define the custom action as follows:

In [3]:
struct OnSign <: Symmetry.OnMonomials end
using PermutationGroups
import SymbolicWedderburn
SymbolicWedderburn.coeff_type(::OnSign) = Float64
function SymbolicWedderburn.action(::OnSign, p::Permutation, mono::AbstractMonomial)
    if isone(p) || iseven(DynamicPolynomials.degree(mono))
        return 1 * mono
    else
        @assert p.perm == perm"(1,2)"
        return -1 * mono
    end
end
G = PermGroup([perm"(1,2)"])

Permutation group on 1 generator generated by
 (1,2)

We can exploit the symmetry as follows:

In [4]:
import CSDP
solver = CSDP.Optimizer
model = Model(solver)
@variable(model, t)
@objective(model, Max, t)
pattern = Symmetry.Pattern(G, OnSign())
con_ref = @constraint(model, poly - t in SOSCone(), symmetry = pattern)
optimize!(model)
value(t)

Stuck at edge of primal feasibility, giving up. 
Partial Success: SDP solved with reduced accuracy
Primal objective value: -9.3197289e-01 
Dual objective value: -9.3290351e-01 
Relative primal infeasibility: 4.73e-08 
Relative dual infeasibility: 6.44e-10 
Real Relative Gap: -3.25e-04 
XZ Relative Gap: 1.04e-08 
DIMACS error measures: 6.21e-08 0.00e+00 1.70e-09 0.00e+00 -3.25e-04 1.04e-08
CSDP 6.2.0
Iter:  0 Ap: 0.00e+00 Pobj:  0.0000000e+00 Ad: 0.00e+00 Dobj:  0.0000000e+00 
Iter:  1 Ap: 9.00e-01 Pobj: -4.4265675e+00 Ad: 9.00e-01 Dobj: -3.0000515e-02 
Iter:  2 Ap: 1.00e+00 Pobj: -4.9278250e+00 Ad: 9.00e-01 Dobj:  1.3441397e+00 
Iter:  3 Ap: 9.00e-01 Pobj: -1.7730372e+00 Ad: 9.00e-01 Dobj: -6.8418354e-01 
Iter:  4 Ap: 9.00e-01 Pobj: -1.1059483e+00 Ad: 9.00e-01 Dobj: -9.8702448e-01 
Iter:  5 Ap: 9.00e-01 Pobj: -1.0107592e+00 Ad: 9.00e-01 Dobj: -9.9870634e-01 
Iter:  6 Ap: 9.00e-01 Pobj: -1.0010797e+00 Ad: 1.00e+00 Dobj: -9.9999427e-01 
Iter:  7 Ap: 9.00e-01 Pobj: -1.0001071e+00 Ad: 1.00

-1.0000000042454928

We indeed find `-1`, let's verify that symmetry was exploited:

In [5]:
gram_matrix(con_ref)

SparseGramMatrix{Float64, FixedPolynomialBasis{DynamicPolynomials.Polynomial{true, Float64}, Vector{DynamicPolynomials.Polynomial{true, Float64}}}, Float64, SymMatrix{Float64}}(GramMatrix{Float64, FixedPolynomialBasis{DynamicPolynomials.Polynomial{true, Float64}, Vector{DynamicPolynomials.Polynomial{true, Float64}}}, Float64, SymMatrix{Float64}}[GramMatrix{Float64, FixedPolynomialBasis{DynamicPolynomials.Polynomial{true, Float64}, Vector{DynamicPolynomials.Polynomial{true, Float64}}}, Float64, SymMatrix{Float64}}([0.999999999999832 -1.0000000011336343; -1.0000000011336343 1.0000000042456882], FixedPolynomialBasis{DynamicPolynomials.Polynomial{true, Float64}, Vector{DynamicPolynomials.Polynomial{true, Float64}}}(DynamicPolynomials.Polynomial{true, Float64}[x², 1.0])), GramMatrix{Float64, FixedPolynomialBasis{DynamicPolynomials.Polynomial{true, Float64}, Vector{DynamicPolynomials.Polynomial{true, Float64}}}, Float64, SymMatrix{Float64}}([2.267148335865912e-9;;], FixedPolynomialBasis{Dyna

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*